##### 1.touch.optim 概述

> 一些前置知识点

- 1）当 requires_grad=True torch 会记录它的计算图并在 parameter.grad 中保存梯度，也就是说 autograd 记录的是涉及带 requires_grad=True 张量的运算，而不是所有运算

- 2）冻结的层的前向计算结果会参与链式求导（用于后面层的梯度计算），但其自身的 parameter.grad 不会被计算和存储，requires_grad=False 冻结这层参数（不求导，不更新）

- 3）loss.backward() 所有可训练参数的 parameter.grad 被填充，即使冻结某层参数（requires_grad=False），在反向传播时误差信号仍会向前传递，用于计算前面层的梯度，但该层的参数不会累积梯度（其 parameter.grad 依旧为 None）

- 4）optimizer.step() 对应的是 param.data -= lr * parameter.grad 手动更新权重这句话，也就是前面那句话的一个高级封装

- 5）冻结层的参数梯度（∂L/∂W）完全不会被计算或缓存，但链式求导中所需的输入梯度（∂L/∂x_in）仍会被计算，以便继续往前传播，要分清参数的梯度和链式传播的输入输出的梯度是不一样的，往前进行链式梯度求导的时候，假如当前层是被冻结的，不需要求他的参数梯度，只需要求他的输入梯度，也就是他的前一层的输出梯度用来前向传播

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TinyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
        self.fc3 = nn.Linear(2, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x)) # layer1
        x = F.relu(self.fc2(x)) # layer2
        x = self.fc3(x) # layer3
        return x

net = TinyNet()

# 冻结中间层 fc2
for p in net.fc2.parameters():
    p.requires_grad = False

# 输入与目标
x = torch.tensor([[1.0, 2.0]])
y_true = torch.tensor([[1.0]])

y_pred = net(x)
loss = F.mse_loss(y_pred, y_true)
loss.backward()

for name, p in net.named_parameters():
    print(name, p.requires_grad, p.grad, p.data) # 可以发现梯度还是会向前传播的，也就是冻结的层也会参与链式法则的计算，只是自己不更新梯度

fc1.weight True tensor([[0., 0.],
        [0., 0.]]) tensor([[-0.4463, -0.3356],
        [-0.6117,  0.4623]])
fc1.bias True tensor([0., 0.]) tensor([0.3516, 0.0590])
fc2.weight False None tensor([[-0.1146,  0.4681],
        [ 0.5340, -0.3494]])
fc2.bias False None tensor([-0.4124, -0.0402])
fc3.weight True tensor([[0., 0.]]) tensor([[ 0.2628, -0.6106]])
fc3.bias True tensor([-3.1934]) tensor([-0.5967])


##### 2.torch.optim.Optimizer 的学习

> 常见的优化器一共可分为两大类，分为无动量类和包含动量的类：SGD、Adagrad、Adam、AdamW、Adamax、SGD + momentum 等

- 1）torch.optim 是整个系统里面唯一修改参数值的模块，接收可训练参数（requires_grad=True）根据梯度 parameter.grad 计算更新规则并修改参数 parameter.data，也就是 parameter.data -= lr * parameter.grad 的高级封装

- 2）所有的优化器都继承自 torch.optim.Optimizer 每个优化器都有这三类成员：

- - 2.1）params：要更新的参数（通常是 model.parameters()）

- - 2.2）state：每个参数的历史状态（比如动量、平方梯度等）

- - 2.3）param_groups：一组参数的配置字典（比如不同层不同学习率，正则项等）

- 3）torch.optim 的所有优化器都是对梯度下降法的封装，他接收一组参数，自动从每个参数的 .grad 里面提取出梯度，根据优化规则更新，优化器除此之外做了三件额外的事情：

- - 3.1）维护历史状态，动量、平方梯度等

- - 3.2）支持不同层的不同学习率

- - 3.2）自动管理梯度清零和数值稳定性

In [6]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import torchvision.transforms as T

class Model(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.backbone = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), stride=1)
        self.head = nn.Linear(in_features=5408, out_features=10)
    def forward(self, x):
        out = self.backbone(x)
        out = out.reshape(512, -1)
        out = self.head(out)
        return out
# ！！！！！训练的时候需要把图片和网络参数显式的放到 GPU 上面，某些在上面某些不在会报错，无法进行训练

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

model = Model(1, 8).to(device)
transform = T.ToTensor() # 训练的时候不转化为 Tensor 会直接报错，因为本身图片是 PIL 类型的
dataset = datasets.MNIST("./data", train=False, transform=transform)
loader = DataLoader(dataset, batch_size=512, shuffle=True, drop_last=True) # 不足丢弃的原因的，输入全连接层之前需要 reshape 到对应的 batch_size 大小，处理起来不方便

# optimizer = torch.optim.Adam(
#     model.parameters(),
#     lr=1e-3, # 学习率
#     betas=(0.9, 0.999), # Adam 特有，代表一阶、二阶的动量系数
#     eps=1e-8, # 防止除 0，数值稳定
#     weight_decay=1e-4, # 权重衰减，权重衰减本质上等价于在损失函数中自动加上一个 L2 正则化项，也就是正则化项的系数
# )

# 可以给不同层不同的学习率
optimizer = torch.optim.Adam([
    {'params': model.backbone.parameters(), 'lr': 1e-4},
    {'params': model.head.parameters(), 'lr': 1e-3}
])

criterion = nn.CrossEntropyLoss()

# 完整的训练过程下，优化器一般的使用如下
for epoch in range(10):
    total_loss = 0
    for datas, labels in loader:
        datas, labels = datas.to(device), labels.to(device) # 数据加载到 GPU
        optimizer.zero_grad() # 清空上次的梯度
        y_pred = model(datas) # 前向传播
        loss = criterion(y_pred, labels)  # 计算损失
        loss.backward() # 反向传播（autograd 填充梯度）
        optimizer.step() # 参数更新
        total_loss += loss.item() # 直接加 loss 的话，他是一个张量，加上之后 total_loss 变成一个张量（计算图会被连起来），autograd 会认为你在构造一个新图，
        # .item() 的作用是：把不需要反向传播的标量张量，安全地转为 Python 数值，仅限标量张量可以这样做
    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch + 1}/10, Loss {avg_loss}")

# .zero_grad() 必须放在 .backward() 前
# .step() 必须放在 .backward() 后

cuda
Epoch 1/10, Loss 1.6250693860806917
Epoch 2/10, Loss 0.8250657225909986
Epoch 3/10, Loss 0.5837368306360746
Epoch 4/10, Loss 0.48924766245641205
Epoch 5/10, Loss 0.43455840098230464
Epoch 6/10, Loss 0.39489366976838364
Epoch 7/10, Loss 0.3688101658695622
Epoch 8/10, Loss 0.3522097327207264
Epoch 9/10, Loss 0.33019823463339554
Epoch 10/10, Loss 0.3149356622444956


##### 3.torch.optim.lr_scheduler 的学习

- 1）在 torch.optim.lr_scheduler 子模块中，可以动态修改学习率，下面是几种常见的学习率调度器：

- - 1.1）StepLR：每隔几步衰减 LR

- - 1.2）MultiStepLR：在指定 epoch 衰减 LR

- - 1.3）ExponentialLR：按指数衰减

- - 1.4）CosineAnnealingLR：余弦退火

- - 1.5）ReduceLROnPlateau：当验证集 loss 不再下降时自动降 LR

- - 1.6）OneCycleLR：Transformer 常用，先升后降

- 2）每个学习率调度器包含的核心方法如下：

- - 2.1）get_lr()：获取当前学习率，是旧接口现在不建议使用了，使用 2.5 的新接口

- - 2.2）step()：执行一次学习率更新（通常每个 epoch 调一次）

- - 2.3）state_dict() / load_state_dict()：保存 / 恢复状态

- - 2.4）last_epoch：当前步数或 epoch 计数，如果断点恢复训练而没加载它，LR 曲线会从头开始，破坏训练计划

- - 2.5）get_last_lr()：获取当前的学习率列表，因为一般情况下设置所有的学习率相同列表里面只包含一个元素，所以直接 get_last_lr()[0] 得到这个值，多个学习率参数的话建议直接输出

- - 2.6）base_lrs：存储初始学习率初始化时记录的每个 param group 的起始 lr）

In [7]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1) # 对当前的优化器的学习率的设置，每十步衰减一次
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 7, 9], gamma=0.1) # 在给定的 epoch 列表（milestones）时刻把 LR 乘 gamma
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95) # 每次 step() 把 LR 乘以常数 gamma
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-6) #在 T_max 次 step（通常=epoch）内从初始 LR 余弦退火到 eta_min
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, mode='min', factor=0.5, patience=3, min_lr=1e-6, verbose=True
#     # min 表示监控 loss，max 表示监控准确率等上升指标
#     # factor 就是其他迭代器的 gamma
#     # patience 连续多少个 epoch 无改善后下降
#     # threshold 小于此阈值代表无改善
#     # min_lr 是 lr 的下限，不能再降低
#     # eps 防止数值抖动
#     # verbose 打印学习率变化日志
# ) # 当验证集指标 metric 多次不改善时，LR 乘 factor

# scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer,
#     max_lr=0.01, # 峰值学习率
#     total_steps=None, # 总步数 = epochs * steps_per_epoch，不提供的话用下面两个指标推算
#     # epochs=EPOCHS,
#     # steps_per_epoch=len(dataloader),
#     pct_start=0.3, # 上升阶段占比（默认30%）
#     anneal_strategy='cos', # 退火策略：'cos' 或 'linear'
#     div_factor=25.0, # 初始 initial = max_lr / div_factor
#     final_div_factor=1e4, # 最终LR = initial_lr / final_div_factor
#     three_phase=False, # 是否三阶段（两个上升阶段 + 一个下降阶段）
#     base_momentum=0.85, # 动量下限
#     max_momentum=0.95, # 动量上限（与LR反向变化）
#     last_epoch=-1,
#     verbose=False # 是否打印日志
# )

# 会继续训练因为模型的参数存在显存里面，还没被释放

for epoch in range(10):
    total_loss = 0
    for datas, labels in loader:
        datas, labels = datas.to(device), labels.to(device) # 数据加载到 GPU
        optimizer.zero_grad() # 清空上次的梯度
        y_pred = model(datas) # 前向传播
        loss = criterion(y_pred, labels)  # 计算损失
        loss.backward() # 反向传播（autograd 填充梯度）
        optimizer.step() # 参数更新
        total_loss += loss.item() # 直接加 loss 的话，他是一个张量，加上之后 total_loss 变成一个张量（计算图会被连起来），autograd 会认为你在构造一个新图，
        # .item() 的作用是：把不需要反向传播的标量张量，安全地转为 Python 数值，仅限标量张量可以这样做

    
    scheduler.step()


    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch + 1}/10, Loss {avg_loss}, lr {scheduler.get_last_lr()}")

Epoch 1/10, Loss 0.30190436306752655, lr [0.0001, 0.001]
Epoch 2/10, Loss 0.29637276969457926, lr [0.0001, 0.001]
Epoch 3/10, Loss 0.2884920921764876, lr [1e-05, 0.0001]
Epoch 4/10, Loss 0.276427099579259, lr [1e-05, 0.0001]
Epoch 5/10, Loss 0.2723829659976457, lr [1e-05, 0.0001]
Epoch 6/10, Loss 0.27412253147677373, lr [1.0000000000000002e-06, 1e-05]
Epoch 7/10, Loss 0.27193330071474375, lr [1.0000000000000002e-06, 1e-05]
Epoch 8/10, Loss 0.2729413760335822, lr [1.0000000000000002e-06, 1e-05]
Epoch 9/10, Loss 0.2725275080454977, lr [1.0000000000000002e-07, 1.0000000000000002e-06]
Epoch 10/10, Loss 0.26978201379901484, lr [1.0000000000000002e-07, 1.0000000000000002e-06]


##### 4.保存整个模型包含优化器、调度器、损失等

In [8]:
# 下面的做法可以在训练中断的时候保存当前的模型

torch.save({
    "model": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "scheduler": scheduler.state_dict(),
    "epoch": epoch, # 这里的 epoch 和 loss 是循环最后一次迭代结束时的值，因为 Python 不会在 for 内创建新的作用域，所以循环结束后，外部依然能访问它们
    "loss": loss
}, "./data/checkpoint.pth")

checkpoint = torch.load("./data/checkpoint.pth", map_location=device)

model.load_state_dict(checkpoint["model"])
optimizer.load_state_dict(checkpoint["optimizer"])
scheduler.load_state_dict(checkpoint["scheduler"])
start_epoch = checkpoint["epoch"] + 1
loss = checkpoint["loss"]

print(epoch, loss, scheduler.get_last_lr())

9 tensor(0.3215, device='cuda:0', requires_grad=True) [1.0000000000000002e-07, 1.0000000000000002e-06]
